# Informacje biznesowe

Kontakt z autorem: [adam_lewandowski_1998@outlook.com](mailto:adam_lewandowski_1998@outlook.com)

Celem byłoby pobranie, poprzez webscrapping (?), kategorii dla przepisów. Wynik mógłby być w postaci np. pliku csv zawierającego wiersze z kolumnami:
1) link (np. https://cookbooks.com/Recipe-Details.aspx?id=434612), który uznajemy za identyfikator przepisu, plus
2) lista kategorii, np. [‘Healthy’, ‘Salad’, ‘Quick and Easy’]
Np. na stronie cookbooks.com mamy po lewej stronie menu z podziałem przepisów na kategorie. Jeśli, przykładowo, klikniemy w Healthy, to dostaniemy listę przepisów, które opisane są taką kategorią.
Pewne utrudnienie: jeśli kliknę w link do takiego przepisu, to (przynajmniej na pierwszy rzut oka) nie mam nigdzie takiego tagu/kategorii. Pewno więc trzeba by było robić odwrotnie: pobierać zbiór przepisów pod daną kategorię i dopiero w drugim kroku zrobić z tego listę linków ze zbiorami kategorii.
Linki miałyby pochodzić jednocześnie np. ze strony Cookbooks.com, ale wyselekcjonowane te, które już są we zbiorze RecipeNLG, bo ten zbiór chcemy wzbogacać. Pewno więc można by zacząć od stworzenia listy linków ze zbioru RecipeNLG, które pochodzą z Cookbooks.com.
Na początek interesująca jest strona Cookbooks.com, ponieważ najwięcej przepisów w zbiorze RecipeNLG pochodzi z tej strony.

# Implementacja

Import bibliotek

In [ ]:
import typing
import requests
from bs4 import BeautifulSoup, Tag
import re
from itables import init_notebook_mode
from tqdm.auto import tqdm
import pandas as pd
from collections import defaultdict
from itertools import chain

init_notebook_mode(all_interactive=True)
BASE_URL = "https://cookbooks.com/"
LINK_TAG = "a"


Nawigacja po stronie wymaga inicjalizacji sesji ze względu na zapisywanie ciasteczek w przestrzeni przeglądarki użytkownika.
Ekstrakcja przepisów wymaga podróżowania bezpośrednio do podkategorii ze względu na wykorzystywanie poprzedniego odwiedzonego adresu do ustalania następnego uwzględniając daną kategorię (pod-strony kategorii współdzielą ten sam adres URL).

In [ ]:
session = requests.Session()


def get_category_recipies(category_url: str) -> list[list[dict]]:
    """
    Extract category recipies from subpages.

    Args:
        category_url (str): category sub-page resource address

    Returns:
        list[list[dict]]: category recipies ids with titles
    """
    TO_REMOVE = re.compile(r"\d+\. ")

    recipies = []
    text = session.get(category_url).content

    html = BeautifulSoup(text, "html.parser")
    links = list(html.find_all("a"))

    recipies.extend(
        [
            {
                "href": f"{BASE_URL}{a['href']}",
                "object": TO_REMOVE.sub("", a.text),
            }
            for a in links
            # url to object
            if a["href"].startswith("Recipe-Details.aspx?")
        ]
    )

    return recipies


def paginate_category_pages(category_url: str):
    """
    Generator to retrieve category subpages urls.

    Args:
        category_url (str): category resource address of recipies

    Yields:
        str: category sub-page resources to visit
    """
    text = session.get(category_url).content

    html = BeautifulSoup(text, "html.parser")
    links = list(html.find_all("a"))

    for a in links:
        if a["href"].startswith("recipes-search.aspx?page"):
            page_url: str = f"{BASE_URL}{a['href']}"
            yield page_url


def get_all_categories() -> dict[str, list[str]]:
    """
    Fetch all categories from page menu, then visits each of them in menu
    order and using page navigation gets links to recipies.

    Returns:
        dict[str, list[str]]: categories with assigned recipies id's in url 
        format
    """
    base_page_html = BeautifulSoup(
        requests.get(BASE_URL).content, "html.parser"
    )
    menu = typing.cast(
        Tag, base_page_html.find("div", attrs={"class": "arrowgreen"})
    )

    recipies = defaultdict(list)
    categories = list(menu.find_all(LINK_TAG))
    for tag in tqdm(categories):
        category = tag.text
        first_page_url = f'{BASE_URL}{tag["href"][3:]}'

        # we have to visit all the pages in a category before moving
        # to the next category
        for url in chain(
            [first_page_url], paginate_category_pages(first_page_url)
        ):
            result = get_category_recipies(url)
            recipies[category].extend(result)

    return recipies



In [ ]:
recipies = get_all_categories()
recipies


Posprocessing

In [ ]:
data = [
    {**el, **{cat: int(el in recipies[cat]) for cat in recipies.keys()}}
    for rec in recipies.values()
    for el in rec
]
df = pd.DataFrame(data).drop_duplicates()
df


In [ ]:
df.to_csv("../out/results_unnormalized.csv", index=False)


Posprocessing dla klasyfikacji

In [ ]:
classification = (
    df.drop(columns=["href"])
    .groupby("object")
    .sum(numeric_only=True)
    .applymap(lambda x: min(x, 1))
)

classification


In [ ]:
classification.to_csv("../out/results_classification.csv", index=True)
